# Axisymmetric Magneto-Couette Flow with Vertical Hydrostatic Balance and Radial Centrifugal-Pressure-Gradient Balance (no meridional flow)

In [ ]:
import dedalus.public as de
from dedalus.tools import post
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
from scipy.special import erf,i1,k1
import scipy.optimize
#import seaborn as sns
import pandas as pd
import glob
import file_tools as flt
import re

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
from matplotlib import font_manager
import matplotlib

font_dirs = ['/Users/cydavid/Documents/Fonts/CMBright']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

plt.rcParams['font.family'] = 'CMU Bright'
plt.rcParams['font.weight'] = 'regular'
plt.rc('axes', unicode_minus=False)

# Implement 2D analytical solution:

In [ ]:
def An(r,n,γ,χ):
    an = (16*(1 + χ)*(i1(((-0.5 + n)*np.pi*r)/γ)*(-(r*k1(((-0.5 + n)*np.pi)/γ)) + χ*r*k1((χ*(-0.5 + n)*np.pi)/γ)) + 
            χ*i1((χ*(-0.5 + n)*np.pi)/γ)*(k1(((-0.5 + n)*np.pi)/γ) - r*k1(((-0.5 + n)*np.pi*r)/γ)) + 
            i1(((-0.5 + n)*np.pi)/γ)*(-(χ*k1((χ*(-0.5 + n)*np.pi)/γ)) + 
            r*k1(((-0.5 + n)*np.pi*r)/γ))))/(χ*(-1 + 2*n)**3*np.pi**3*r*(i1((χ*(-0.5 + n)*np.pi)/γ)*k1(((-0.5 + n)*np.pi)/γ) - 
            i1(((-0.5 + n)*np.pi)/γ)*k1((χ*(-0.5 + n)*np.pi)/γ)))
    return an
        
def u2D(r,z,l,γ,χ):
    u2Darr = np.array([An(r,n,γ,χ)*np.sin((-0.5 + n)*np.pi*z) for n in range(1,l+1)])
    return u2Darr.sum(axis=0)

# Implement solver for magneto-Couette flow

In [ ]:
def MagnetoCouetteFlow(ReC,γ,χ,simtime,save_freq,dt,nz,nr,timestepper,save_dir,Print=True):
    split = 1

    max_sim_time = simtime
    save_freq = int(round(save_freq))
    
    zbasis = de.SinCos('z',nz,interval=(-.1,1))
    rbasis = de.Chebyshev('r',nr,interval=(χ,1))
    domain = de.Domain([zbasis,rbasis],grid_dtype=np.float64)
    z, r = domain.grids()

    mask = lambda x: (1/2)*(1 - erf(np.pi**(1/2)*x))
    δ = .05
    A = 3.11346786
    wall = domain.new_field()
    wall.meta['z']['parity'] = 1
    wall['g'] = mask(z/δ)
    ϵ = δ/A
    Γ = 1/ReC*ϵ**(-2)

    switch = domain.new_field()
    switch.meta['z']['parity'] = -1
    switch['g'] = np.tanh(-(z-1)/δ) - wall['g']


    def heaviside(x): 
        if x < 0: return 0
        elif x >=0: return 1

    from dedalus.core.operators import GeneralFunction
    # Define GeneralFunction subclass for time dependent boundary conditions
    class ConstantFunction(GeneralFunction):
        def __init__(self, domain, layout, func, args=[], kw={}, out=None,):
            super().__init__(domain, layout, func, args=[], kw={}, out=None,)

        def meta_constant(self, axis):
            return True

        def meta_parity(self, axis):
            return 1

    def impulse(solver): return heaviside(solver.sim_time)

    impulse_func = ConstantFunction(domain, layout='g', func=impulse)

    problem = de.IVP(domain,variables=['ut','utr','Theta'])
    problem.parameters['ReC'] = ReC
    problem.parameters['wall'] = wall
    problem.parameters['γ'] = γ
    problem.parameters['χ'] = χ
    problem.parameters['Γ'] = Γ
    problem.parameters['switch'] = switch
    problem.parameters['i'] = impulse_func
    problem.parameters['A'] = split
    
    problem.meta['ut','utr','Theta']['z']['parity'] = 1 # cosines
    
    problem.add_equation('dr(ut) - utr = 0')
    problem.add_equation('dt(ut) - 1/ReC*(γ**2*dr(utr) + dz(dz(ut))) + A*Γ*ut =  γ**2/ReC*(utr/r - ut/r**2) + (1+χ)/ReC*i/r + Γ*(A-wall)*ut')
    problem.add_equation('dt(Theta) - (1+χ)*ut/r = 0')
    
    problem.add_bc('left(ut) = 0')
    problem.add_bc('right(ut) = 0')

    solver = problem.build_solver(getattr(de.timesteppers,timestepper))

    impulse_func.original_args = impulse_func.args = [solver]

    zz, rr = z+0*r, 0*z+r

    ut, utr, Theta = [solver.state[_] for _ in problem.variables]

    import file_tools as flt # another github repo of Eric Hester's on github
    flt.makedir(save_dir)
    analysis = solver.evaluator.add_file_handler(f'{save_dir}/analysis-{sim_name}',iter=save_freq)
    for f in problem.variables: analysis.add_task(f)

    while solver.sim_time <= simtime:
        solver.step(dt)
        if solver.iteration % 100 == 0:
            if Print==True:
                print(f'it {solver.iteration}',
                      f't {solver.sim_time:.2f}',
                      f'|ut| max {np.abs(ut["g"]).max():.3f}')
            if np.any(np.isnan(ut['g'])): 
                print('Broken')
                break

# Matching experiments with 2D axisymmetric simulations

In [ ]:
nondim_exp_param = pd.read_csv('../experimental_processing/experimental_data/nondim_exp_param.csv')
nondim_exp_param.head(2)

In [ ]:
runs = runs = np.array(['RUN A3','RUN C2','RUN D1','RUN E1'])

In [ ]:
save_dir = 'data/CentHSE/exp-matching'

## Simulate

In [ ]:
%%time

for run in runs:
    ReC = nondim_exp_param[nondim_exp_param['RUN']==run]['ReC'].item()
    γ = nondim_exp_param[nondim_exp_param['RUN']==run]['γ'].item()
    χ = nondim_exp_param[nondim_exp_param['RUN']==run]['χ'].item()
    
    τ = (2/np.pi)**2*ReC
    simtime = 1.1*np.pi #5*τ #2*5*τ
    save_freq = 2
    timestepper = 'SBDF2'
    
    dt = 5e-5*simtime #2.5e-4*simtime
    nz = 100 #20
    nr = 201 #500 #50

    run_name = run.replace(' ','-')
    sim_name = f'mhd-2D-axisymmetric-VP-implicit-{timestepper}-{run_name}-nz={nz}-nr={nr}'
    print(sim_name)
    
    ###############

    MagnetoCouetteFlow(ReC,γ,χ,simtime,save_freq,dt,nz,nr,timestepper,save_dir,Print=False)
    

## Analysis

### Merge output files and get keys

In [ ]:
# Get sorted list of directories containing data files
analysis_dirs = sorted(glob.glob(f'{save_dir}/analysis*VP*'))
# Merge output files
for analysis in analysis_dirs:
    post.merge_analysis(analysis,cleanup=True)
    
    # Get list of HDF5 set paths
    setpaths = glob.glob(f'{analysis}/*s?.h5')
    if len(setpaths)!=0:
        jointpath = re.sub('_s*.', '', setpaths[0])
        post.merge_sets(jointpath, setpaths, cleanup=True)

### Load data

In [ ]:
runs

In [ ]:
fig, axs = plt.subplots(1,len(runs))
i = 0
for run in runs:
    run_name = run.replace(' ','-')
    i = np.where(runs==run)[0][0]
    files = glob.glob(f'{save_dir}/*/*{run_name}*.h5')
    for file in files:
        r, z, t = flt.load_data(file,'r/1.0','z/1.0','sim_time',group='scales')
        zz, rr = np.meshgrid(z, r, indexing='ij')
        uts, = flt.load_data(file, 'ut',group='tasks')
        Thetas, = flt.load_data(file, 'Theta',group='tasks')
        axs[i].plot(rr[-1,:],uts[-1,-1,:])
    i+=1

### Export angular displacement data

In [ ]:
surfangdisp = Thetas[:,-1,:]

dye_advec_save_dir = f'{save_dir}/DyeAdvection'
flt.makedir(dye_advec_save_dir)

angdispfname = f'{dye_advec_save_dir}/Ang-Disp-{run_name}.csv'
np.savetxt(angdispfname,surfangdisp,delimiter=',')

tfname = f'{dye_advec_save_dir}/simtimes-{run_name}.csv'
np.savetxt(tfname,t,delimiter=',')

rfname = f'{dye_advec_save_dir}/radii-{run_name}.csv'
np.savetxt(rfname,r,delimiter=',')

### Plot azimuthal flow

In [ ]:
tindex = -1

flt.makedir('figures/exp-matching')
fig, ax = plt.subplots()

clim = np.array([0,umax])
ps = ax.pcolormesh(rr, zz, uts[tindex,:,:],shading='auto',cmap='RdBu_r',vmin=clim)
ps.set_edgecolor('face')
plt.colorbar(ps,ax=ax,label='$\\tilde{u}_{\\theta}$')
q=ax.quiver(rr[::1,::20],zz[::1,::20],urs[tindex,::1,::20],uzs[tindex,::1,::20])
ax.quiverkey(q,X=0.8, Y=1.05, U=umerMG,
             label='$||(\\tilde{u}_r,\\tilde{u}_z)|| =$'+f'{umerMG:.3f}', labelpos='E')
ax.set(ylim=[0,1])
ax.set_xlabel('$\\tilde{r}$')
ax.set_ylabel('$\\tilde{z}$')
ax.set_title('Flow field for '+run)

ax.annotate('inner cylinder', xy=(0, -0.15),  xycoords='axes fraction', xytext=(30, -4), 
            textcoords='offset points', arrowprops=dict(arrowstyle="->"), annotation_clip=False)
ax.annotate('outer cylinder', xy=(1, -0.15),  xycoords='axes fraction', xytext=(-100, -4), 
            textcoords='offset points', arrowprops=dict(arrowstyle="->"), annotation_clip=False)
plt.tight_layout()
plt.savefig(f'figures/exp-matching/flowfield-{run_name}.pdf')

### Compare to experimental data

In [ ]:
u_df = pd.read_csv('../experimental_processing/experimental_data/vel.csv')
colors = np.loadtxt("../experimental_processing/colors.txt",dtype=str,comments='%')
Uscale = nondim_exp_param[nondim_exp_param['RUN']==run]['U, MHD (m/s)'].item()
nd_u = u_df[run]/Uscale
r_o = 0.0975
nd_r = u_df['r (m)']/r_o
ReC = nondim_exp_param[nondim_exp_param['RUN']==run]['ReC'].item()
γ = nondim_exp_param[nondim_exp_param['RUN']==run]['γ'].item()
χ = nondim_exp_param[nondim_exp_param['RUN']==run]['χ'].item()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(nd_r,nd_u,label='experimental data',color=colors[i])

ax.plot(rr[-1],np.abs(uts[tindex,-1,:]),color=colors[i],label='2D axisymmetric simulation')
ax.plot(rr[-1],(1+χ)/2*1/rr[-1],linestyle='--',color='gray',label='$\\frac{1+\\chi}{2}\\frac{1}{\\tilde{r}}$')
ax.plot(rr[-1,:],u2D(rr[-1,:],z=1,l=5,γ=γ,χ=χ),color=colors[i],linestyle='--',label='Low-Re (2D) theory')

ax.set_xlim(χ,1)
ax.set_ylim(0,)
#t_over_tau = np.round(5*(frame_num+1)/np.shape(uts[:,-1,:])[0],1)
ax.set_title(f'Nondimensionalized surface azimuthal velocity\nvs. radius for {run} ($Re_L=${ReC:.2f}, $\\gamma=${γ:.2f}, $\chi=${χ:.2f})')
ax.set_xlabel('$\\tilde{r}$')
ax.set_ylabel('$\\tilde{u}_\\theta (\\tilde{r},\\tilde{z}=1,\\tilde{t})$')
ax.legend()
ax.annotate('inner cylinder', xy=(0, -0.15),  xycoords='axes fraction', xytext=(30, -4), 
            textcoords='offset points', arrowprops=dict(arrowstyle="->"), annotation_clip=False)
ax.annotate('outer cylinder', xy=(1, -0.15),  xycoords='axes fraction', xytext=(-100, -4), 
            textcoords='offset points', arrowprops=dict(arrowstyle="->"), annotation_clip=False)
plt.tight_layout()
plt.savefig(f'figures/exp-matching/{run_name}.pdf')
plt.show()

# Compare to 2D solution

In [ ]:
γ_arr = np.linspace(0.01,0.06,2)
ReC = 0.1
χ_arr = np.linspace(0.1,0.6,2) 
ReC_str = f'{ReC:.2f}'.replace('.','_')
save_dir_SL = f'data/CentHSE/comparison/ReC{ReC_str}'

## Simulate

In [ ]:
%%time
%%capture
for j in np.arange(len(χ_arr)):
    χ = χ_arr[j]
    χ_str = f'{χ:.2f}'.replace('.','_')
    for i in np.arange(len(γ_arr)):
        γ = γ_arr[i]    
        τ = (2/np.pi)**2*ReC
        simtime = 2*5*τ
        save_freq = 200
        timestepper = 'SBDF2'
        
        γ_str = f'{γ:.2f}'.replace('.','_')
        sim_name = f'mhd-2D-axisymmetric-VP-implicit-{timestepper}-ReC{ReC_str}-γ{γ_str}-χ{χ_str}'
        print(sim_name)
        
        nz = 15#40
        nr = 50#500 #1000
        dt = 2.5e-4*simtime
        
        
        ###############

        MagnetoCouetteFlow(ReC,γ,χ,simtime,save_freq,dt,nz,nr,timestepper,save_dir_SL,Print=False)
        

In [ ]:
# Get sorted list of directories containing data files
analysis_dirs = sorted(glob.glob(f'{save_dir_SL}/analysis*VP*'))
# Merge output files
for analysis in analysis_dirs:
    post.merge_analysis(analysis,cleanup=True)
# Get list of HDF5 files
files = [glob.glob(f'{analysis}/*.h5')[-1] for analysis in analysis_dirs]
# Print task keys for data file
flt.print_keys(files[0],group='tasks')

## Plot against 2D solution

In [ ]:
theme = 'DarkBeachColors'
colorfname = glob.glob(f'../ColorMaps/**/{theme}*',recursive=True)[0]
cmaphex = np.loadtxt(colorfname,dtype=str,delimiter=',',comments='%')
cmap = get_continuous_cmap(cmaphex)

In [ ]:
file=glob.glob(f'{save_dir_SL}/*/*-ReC{ReC_str}-γ{γ_str}-χ{χ_str}*.h5')[0]

In [ ]:
flt.makedir(f'figures/Comparisonto2D')
l=2
for j in np.arange(len(χ_arr)):
    χ = χ_arr[j]
    χ_str = f'{χ:.2f}'.replace('.','_')

    fig, ax = plt.subplots(figsize=(7.25,4))

    for i in np.arange(len(γ_arr)):
        γ = γ_arr[i]
        γ_str = f'{γ:.2f}'.replace('.','_')
        file = glob.glob(f'{save_dir_SL}/*/*-ReC{ReC_str}-γ{γ_str}-χ{χ_str}*.h5')[0]
        r, z, t = flt.load_data(file,'r/1.0','z/1.0','sim_time',group='scales')
        zz, rr = np.meshgrid(z, r, indexing='ij')
        uts, = flt.load_data(file, 'ut',group='tasks')

        # Plot
        ax.plot(rr[-1,:],uts[-1,-1,:],color=cmap(i/len(γ_arr)),label=f'$\gamma=${γ:.2f}')
        ax.plot(rr[-1,:],u2D(rr[-1,:],z=1,l=l,γ=γ,χ=χ),color=cmap(i/len(γ_arr)),linestyle='--')
        
    ax.plot(-1,-1,linestyle='--',color='gray',label=f'low-Re 2D\ntheory ($l=${l})')
    ax.set_xlim(χ,1)
    ax.set_ylim(0,)
    ax.set_title(f'Surface azimuthal velocity vs. radius, $Re_L=${ReC}, $\chi=${χ:.2f}') #at $\\tilde{t}$='+str(t_over_tau)+'$\\tau$')
    ax.set_xlabel('$\\tilde{r}$')
    ax.set_ylabel('$\\tilde{u}_\\theta (\\tilde{r},\\tilde{z}=1,\\tilde{t})$')
    ax.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    ax.annotate('inner cylinder', xy=(0, -0.15),  xycoords='axes fraction', xytext=(30, -4), 
                textcoords='offset points', arrowprops=dict(arrowstyle="->"), annotation_clip=False)
    ax.annotate('outer cylinder', xy=(1, -0.15),  xycoords='axes fraction', xytext=(-100, -4), 
                textcoords='offset points', arrowprops=dict(arrowstyle="->"), annotation_clip=False)
    plt.tight_layout()
    
    plt.savefig(f'figures/Comparisonto2D/ReC{ReC_str}-χ{χ_str}.pdf')